In [1]:
import torch
import os
from mujoco_physics import HopperPhysics
import cv2
import glob

# Rendering the Ground Truth

Enter the path to the pt file here in the data_path variable

In [2]:
data_path= r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228\data\HopperPhysics\training.pt"

In [3]:
dataset = torch.load(data_path)
print("Dataset loaded successfully!")

Dataset loaded successfully!


In [4]:
output_dir_gt = 'ground_truth'
os.makedirs(output_dir_gt, exist_ok=True)

In [5]:
hopper = HopperPhysics(root='data', download=False, generate=False)

Select a trajectory to render (variable: traj_index)

In [6]:
traj_index = 10
trajectory =dataset[traj_index]
if not isinstance(trajectory, torch.Tensor):
    trajectory = torch.Tensor(trajectory)
print(f"Selected trajectory index: {traj_index}")

Selected trajectory index: 10


In [7]:
hopper.visualize(trajectory, plot_name=f'traj_{traj_index}_true', dirname=output_dir_gt)
print(f"Trajectory {traj_index} rendered and saved in {output_dir_gt}")

Trajectory 10 rendered and saved in ground_truth


In [2]:
def frames_to_video(frames_dir, output_video_path, fps=30):
    frames = sorted(glob.glob(os.path.join(frames_dir, "*.jpg")))

    frame = cv2.imread(frames[0])
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for frame_file in frames:
        frame = cv2.imread(frame_file)
        video.write(frame)

    video.release()

    print(f"Video saved as {output_video_path}")

In [10]:
output_vid_gt=r"ground_truth/ground_truth.mp4"
frames_to_video(output_dir_gt,output_vid_gt)

Video saved as ground_truth/ground_truth.mp4


# Training the Model

In [3]:
import torch.optim as optim
from torch.distributions.normal import Normal
import numpy as np
import time 
from random import SystemRandom

import lib.utils as utils
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.utils import compute_loss_all_batches, get_next_batch, makedirs, get_logger

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics
from lib.latent_ode import LatentODE


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

Make changes to all the input args here

In [26]:
args = Args(
    n=10000,  # Size of the dataset
    niters=10,
    lr=1e-2,  # Starting learning rate
    batch_size=50,
    viz=False,  # Show plots while training
    save='experiments/',  # Path to save checkpoints
    load=None,  # ID of the experiment to load for evaluation. If None, run a new experiment
    random_seed=1991,  # Random seed
    dataset='hopper',  # Dataset to load
    sample_tp=None,  # Number of time points to sub-sample
    cut_tp=None,  # Cut out the section of the timeline
    quantization=0.1,  # Quantization on the physionet dataset
    latent_ode=False,  # Run Latent ODE seq2seq model
    z0_encoder='odernn',  # Type of encoder for Latent ODE model
    classic_rnn=True,  # Run RNN baseline
    rnn_cell="expdecay",  # RNN Cell type #gru, expdecay- gru-d
    input_decay=True,  # For RNN: use the input that is the weighted average of empirical mean and previous value
    ode_rnn=True,  # Run ODE-RNN baseline
    rnn_vae=False,  # Run RNN baseline: seq2seq model with sampling of the h0 and ELBO loss
    latents=15,  # Size of the latent state
    rec_dims=30,  # Dimensionality of the recognition model
    rec_layers=3,  # Number of layers in ODE func in recognition ODE
    gen_layers=3,  # Number of layers in ODE func in generative ODE
    units=300,  # Number of units per layer in ODE func
    gru_units=100,  # Number of units per layer in each of GRU update networks
    poisson=False,  # Model poisson-process likelihood for the density of events in addition to reconstruction
    classif=False,  # Include binary classification loss
    linear_classif=False,  # Use a linear classifier instead of 1-layer NN
    extrap=False,  # Set extrapolation mode
    timepoints=100,  # Total number of time-points
    max_t=5.0,  # Subsample points in the interval [0, args.max_t]
    noise_weight=0.01  # Noise amplitude for generated trajectories
)
file_name = "run_models"
makedirs(args.save)

In [7]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

experimentID = args.load
if experimentID is None:
	# Make a new experiment ID
	experimentID = int(SystemRandom().random()*100000)
ckpt_path = os.path.join(args.save, "experiment_" + str(experimentID) + '.ckpt')

start = time.time()
print("Sampling dataset of {} training examples".format(args.n))

input_command = f"run_models.py --n {args.n} --niters {args.niters} --lr {args.lr} --batch_size {args.batch_size} " \
                f"--viz {args.viz} --save {args.save} --random_seed {args.random_seed} --dataset {args.dataset} " \
                f"--latent_ode {args.latent_ode} --classic_rnn {args.classic_rnn} --ode_rnn {args.ode_rnn}--z0_encoder {args.z0_encoder} --latents {args.latents} " \
                f"--rec_dims {args.rec_dims} --rec_layers {args.rec_layers} --gen_layers {args.gen_layers} " \
                f"--units {args.units} --gru_units {args.gru_units} --timepoints {args.timepoints} --max_t {args.max_t} " \
                f"--noise_weight {args.noise_weight} --extrap {args.extrap} "

if args.load:
	input_command += f" --load {args.load}"

makedirs("results/")
    
print(f"Checkpoint path: {ckpt_path}")
print(f"Time taken for setup: {time.time() - start} seconds")
print(f"Input command: {input_command}")

Sampling dataset of 10000 training examples
Checkpoint path: experiments/experiment_75755.ckpt
Time taken for setup: 0.0009975433349609375 seconds
Input command: run_models.py --n 10000 --niters 10 --lr 0.01 --batch_size 50 --viz False --save experiments/ --random_seed 1991 --dataset hopper --latent_ode False --classic_rnn False --ode_rnn True--z0_encoder odernn --latents 15 --rec_dims 30 --rec_layers 3 --gen_layers 3 --units 300 --gru_units 100 --timepoints 100 --max_t 5.0 --noise_weight 0.01 --extrap False 


In [8]:
data_obj = parse_datasets(args, device)
input_dim = data_obj["input_dim"]
	
print(f"Input dimension: {input_dim}")

classif_per_tp = False
if ("classif_per_tp" in data_obj):
		# do classification per time point rather than on a time series as a whole
		classif_per_tp = data_obj["classif_per_tp"]

if args.classif and (args.dataset == "hopper" or args.dataset == "periodic"):
		raise Exception("Classification task is not available for MuJoCo and 1d datasets")

n_labels = 1
if args.classif:
	if ("n_labels" in data_obj):
		n_labels = data_obj["n_labels"]
	else:
		raise Exception("Please provide number of labels for classification task")

Input dimension: 14


In [9]:
obsrv_std = 1e-3 
obsrv_std = torch.Tensor([obsrv_std]).to(device)
z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))

# Model Initialization

In [27]:
if args.rnn_vae:
		if args.poisson:
			print("Poisson process likelihood not implemented for RNN-VAE: ignoring --poisson")

		# Create RNN-VAE model
		model = RNN_VAE(input_dim, args.latents, 
			device = device, 
			rec_dims = args.rec_dims, 
			concat_mask = True, 
			obsrv_std = obsrv_std,
			z0_prior = z0_prior,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			linear_classifier = args.linear_classif,
			n_units = args.units,
			input_space_decay = args.input_decay,
			cell = args.rnn_cell,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.classic_rnn:
		if args.poisson:
			print("Poisson process likelihood not implemented for RNN: ignoring --poisson")

		if args.extrap:
			raise Exception("Extrapolation for standard RNN not implemented")
		# Create RNN model
		model = Classic_RNN(input_dim, args.latents, device, 
			concat_mask = True, obsrv_std = obsrv_std,
			n_units = args.units,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			linear_classifier = args.linear_classif,
			input_space_decay = args.input_decay,
			cell = args.rnn_cell,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.ode_rnn:
		# Create ODE-GRU model
		n_ode_gru_dims = args.latents
				
		if args.poisson:
			print("Poisson process likelihood not implemented for ODE-RNN: ignoring --poisson")

		if args.extrap:
			raise Exception("Extrapolation for ODE-RNN not implemented")

		ode_func_net = utils.create_net(n_ode_gru_dims, n_ode_gru_dims, 
			n_layers = args.rec_layers, n_units = args.units, nonlinear = nn.Tanh)

		rec_ode_func = ODEFunc(
			input_dim = input_dim, 
			latent_dim = n_ode_gru_dims,
			ode_func_net = ode_func_net,
			device = device).to(device)

		z0_diffeq_solver = DiffeqSolver(input_dim, rec_ode_func, "euler", args.latents, 
			odeint_rtol = 1e-3, odeint_atol = 1e-4, device = device)
	
		model = ODE_RNN(input_dim, n_ode_gru_dims, device = device, 
			z0_diffeq_solver = z0_diffeq_solver, n_gru_units = args.gru_units,
			concat_mask = True, obsrv_std = obsrv_std,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.latent_ode:
		model = create_LatentODE_model(args, input_dim, z0_prior, obsrv_std, device, 
			classif_per_tp = classif_per_tp,
			n_labels = n_labels)
else:
	raise Exception("Model not specified")

Change the path according to your environment (script_path variable) (Strictly run this only once)

In [11]:
log_path = "logs/" + file_name + "_" + str(experimentID) + ".log"
if not os.path.exists("logs/"):
	utils.makedirs("logs/")
script_path = os.path.abspath(r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228") 

logger = get_logger(logpath=log_path, filepath=script_path)
logger.info(input_command)

C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228
run_models.py --n 10000 --niters 10 --lr 0.01 --batch_size 50 --viz False --save experiments/ --random_seed 1991 --dataset hopper --latent_ode False --classic_rnn False --ode_rnn True--z0_encoder odernn --latents 15 --rec_dims 30 --rec_layers 3 --gen_layers 3 --units 300 --gru_units 100 --timepoints 100 --max_t 5.0 --noise_weight 0.01 --extrap False 


In [12]:
optimizer = optim.Adamax(model.parameters(), lr=args.lr)
num_batches = data_obj["n_train_batches"]

In [13]:
for itr in range(1, num_batches * (args.niters + 1)):
		optimizer.zero_grad()
		utils.update_learning_rate(optimizer, decay_rate = 0.999, lowest = args.lr / 10)

		wait_until_kl_inc = 10
		if itr // num_batches < wait_until_kl_inc:
			kl_coef = 0.
		else:
			kl_coef = (1-0.99** (itr // num_batches - wait_until_kl_inc))

		batch_dict = utils.get_next_batch(data_obj["train_dataloader"])

		train_res = model.compute_all_losses(batch_dict, n_traj_samples = 3, kl_coef = kl_coef)
		train_res["loss"].backward()
		optimizer.step()

		n_iters_to_viz = 1
		if itr % (n_iters_to_viz * num_batches) == 0:
			with torch.no_grad():

				test_res = compute_loss_all_batches(model, 
					data_obj["test_dataloader"], args,
					n_batches = data_obj["n_test_batches"],
					experimentID = experimentID,
					device = device,
					n_traj_samples = 3, kl_coef = kl_coef)

				message = 'Epoch {:04d} [Test seq (cond on sampled tp)] | Loss {:.6f} | Likelihood {:.6f} | KL fp {:.4f} | FP STD {:.4f}|'.format(
					itr//num_batches, 
					test_res["loss"].detach(), test_res["likelihood"].detach(), 
					test_res["kl_first_p"], test_res["std_first_p"])
		 	
				logger.info("Experiment " + str(experimentID))
				logger.info(message)
				logger.info("KL coef: {}".format(kl_coef))
				logger.info("Train loss (one batch): {}".format(train_res["loss"].detach()))
				logger.info("Train CE loss (one batch): {}".format(train_res["ce_loss"].detach()))
				
				if "auc" in test_res:
					logger.info("Classification AUC (TEST): {:.4f}".format(test_res["auc"]))

				if "mse" in test_res:
					logger.info("Test MSE: {:.4f}".format(test_res["mse"]))

				if "accuracy" in train_res:
					logger.info("Classification accuracy (TRAIN): {:.4f}".format(train_res["accuracy"]))

				if "accuracy" in test_res:
					logger.info("Classification accuracy (TEST): {:.4f}".format(test_res["accuracy"]))

				if "pois_likelihood" in test_res:
					logger.info("Poisson likelihood: {}".format(test_res["pois_likelihood"]))

				if "ce_loss" in test_res:
					logger.info("CE loss: {}".format(test_res["ce_loss"]))

			torch.save({
				'args': args,
				'state_dict': model.state_dict(),
			}, ckpt_path)
torch.save({
    'args': args,
    'state_dict': model.state_dict(),
}, ckpt_path)

print("Training complete. Model saved.")


c:\Users\msi\anaconda3\envs\py310\lib\site-packages\torchdiffeq\_impl\misc.py:296: UserWarning: t is not on the same device as y0. Coercing to y0.device.
  warnings.warn("t is not on the same device as y0. Coercing to y0.device.")


Computing loss... 0


Experiment 75755
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 8658.968750 | Likelihood -8658.968750 | KL fp 0.0000 | FP STD 0.0000|
KL coef: 0.0
Train loss (one batch): 8844.4931640625
Train CE loss (one batch): 0.0
Test MSE: 0.0173
Poisson likelihood: 0.0
CE loss: 0.0


Computing loss... 0


Experiment 75755
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 2764.717529 | Likelihood -2764.717529 | KL fp 0.0000 | FP STD 0.0000|
KL coef: 0.0
Train loss (one batch): 2352.63818359375
Train CE loss (one batch): 0.0
Test MSE: 0.0055
Poisson likelihood: 0.0
CE loss: 0.0


...

Computing loss... 0


Experiment 75755
Epoch 0009 [Test seq (cond on sampled tp)] | Loss 481.514923 | Likelihood -481.514923 | KL fp 0.0000 | FP STD 0.0000|
KL coef: 0.0
Train loss (one batch): 404.9633483886719
Train CE loss (one batch): 0.0
Test MSE: 0.0010
Poisson likelihood: 0.0
CE loss: 0.0


Computing loss... 0


Experiment 75755
Epoch 0010 [Test seq (cond on sampled tp)] | Loss 456.818695 | Likelihood -456.818695 | KL fp 0.0000 | FP STD 0.0000|
KL coef: 0.0
Train loss (one batch): 384.8369445800781
Train CE loss (one batch): 0.0
Test MSE: 0.0009
Poisson likelihood: 0.0
CE loss: 0.0


Training complete. Model saved.


# Loading the trained model for testing

In [28]:
# ckpt_path = r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228\experiments\experiment_64004.ckpt" # with 100 datapoints (takes 3 mins for 100 iters)
# ckpt_path = r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228\experiments\experiment_27160.ckpt" #with 100 datapoints (hopper)
#34851 - 10000 datapoints -less epochs
# ckpt_path =r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228\experiments\experiment_34851.ckpt"
# 10006 - 100 epochs -all points
# ckpt_path = r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228\experiments\experiment_10006.ckpt"
# 1310 - 100 epochs - 10%
# ckpt_path=r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228\experiments\experiment_1310.ckpt"
#36227 - 10 epochs - None- gru-d
# ckpt_path=r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228\experiments\experiment_36227.ckpt"
#75755 - 10 epochs - None- ode-rnn
ckpt_path=r"C:\Users\msi\Desktop\ECE-228\Project\latent_ode_ece_228\experiments\experiment_75755.ckpt"

In [29]:
utils.get_ckpt_model(ckpt_path, model, device)

In [30]:
test_dict = utils.get_next_batch(data_obj["test_dataloader"])

In [31]:
data =  test_dict["data_to_predict"]
time_steps = test_dict["tp_to_predict"]
mask = test_dict["mask_predicted_data"]
		
observed_data =  test_dict["observed_data"]
observed_time_steps = test_dict["observed_tp"]
observed_mask = test_dict["observed_mask"]

device = utils.get_device(time_steps)

time_steps_to_predict = time_steps


if isinstance(model, LatentODE):
	# sample at the original time points
	time_steps_to_predict = utils.linspace_vector(time_steps[0], time_steps[-1], 100).to(device)

reconstructions, info = model.get_reconstruction(time_steps_to_predict, 
	observed_data, observed_time_steps, mask = observed_mask, n_traj_samples = 10)

In [18]:
observed_data.shape

torch.Size([2000, 100, 14])

In [19]:
reconstructions.shape


torch.Size([1, 2000, 100, 14])

In [20]:
reconstructions.mean(dim=0).detach().shape

torch.Size([2000, 100, 14])

Enter your trajectory index

In [32]:
traj_index=1
percentage=None
model="gru-d"

In [33]:
output_dir= f"gt_render_{traj_index}_{percentage}_{model}"
hopper = HopperPhysics(root='data', download=False, generate=False)
hopper.visualize(observed_data[traj_index], plot_name=f'traj_{traj_index}', dirname=output_dir)
output_vid_gt=f"{output_dir}/ground_truth.mp4"
frames_to_video(output_dir,output_vid_gt)

Video saved as gt_render_1_None_gru-d/ground_truth.mp4


In [34]:
output_dir= f"pred_render_{traj_index}_{percentage}_{model}"
hopper.visualize(reconstructions.mean(dim=0)[traj_index].detach(), plot_name=f'traj_{traj_index}', dirname=output_dir)
output_vid_gt=f"{output_dir}/predicted.mp4"
frames_to_video(output_dir,output_vid_gt)

Video saved as pred_render_1_None_gru-d/predicted.mp4
